In [9]:
import os
import datetime
import timedelta
import pandas as pd
import urllib.request, json
from UpdateDerivedValues import updateDerivedValues
# updateDerivedValues


date = (datetime.datetime.now() - timedelta.Timedelta(days=1)).date()
# present = datetime.datetime.now().date()

folders = ["INPUT","RAWCSV","LOG"]
for folder in folders:
    if not os.path.isdir(os.path.join("..","..",folder,str(date))):
        os.mkdir(os.path.join("..","..",folder,str(date)))
        print("Created")

In [10]:
source = pd.read_csv("../../sources.csv")

In [11]:
def getPreviousRawFile(date,state):
    prevDate = date - timedelta.Timedelta(days=1)
    try:
        state_df = pd.read_csv("../../RAWCSV/"+str(prevDate)+"/"+state+"_final.csv")
    except:
        state_df = getPreviousRawFile(prevDate,state)
    return state_df

In [12]:
path = "../../RAWCSV/"+str(date)+"/"
dir_list = os.listdir(path)
for state in source["StateCode"]:
    if state != "TT":
        if state+"_raw.csv" not in dir_list:
            df = getPreviousRawFile(date,state)
            df["Date"] = str(date)
            df.to_csv(path+state+"_final.csv",index=False)
        else:
            updateDerivedValues(state,str(date))

Other States population numbers not found
Other Countries population numbers not found
Running 7DMA for AP
Other
Kra Daadi population numbers not found
Lepa Rada population numbers not found
Pakke Kessang population numbers not found
Running 7DMA for AR
Other
Running 7DMA for BR
Other
Gaurela Pendra Marwahi population numbers not found
Other State population numbers not found
Running 7DMA for CT
Other


/home/swiadmin/Incovid19/incovid19/src/Interim_update/get7DMA.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['7DmaTestedForDistrict'][df['District'] != 'Unknown'] = test_df['7DmaTestedForDistrict'][test_df['District'] != 'Unknown']


Running 7DMA for HR
Other
Running 7DMA for HP
Other
Other State population numbers not found
Running 7DMA for KA
Other
Running 7DMA for KL
Other
Kangpokpi population numbers not found
Noney population numbers not found
Tengnoupal population numbers not found
Running 7DMA for MN
Other
Ri-Bhoi population numbers not found
Running 7DMA for ML
Other
Running 7DMA for OR
Other
Other State population numbers not found
Running 7DMA for RJ
Other
Running 7DMA for TN
Other
Running 7DMA for UT
Other
Other State population numbers not found
Running 7DMA for WB
Other
Running 7DMA for JK
Other
Running 7DMA for PY
Other


In [13]:
df_intTT = pd.read_csv(path+"TT_raw.csv")

In [14]:
df_intTT["Date"] = str(date)
df_intTT["tested_source_state"] = "https://www.icmr.gov.in/"

In [15]:
value = ["Confirmed","Deceased","Recovered"]
for idx in df_intTT.index:
    stateCode = source[source['StateName'] == df_intTT["District"][idx]]["StateCode"].item()
    df_state = pd.read_csv(path + stateCode + "_final.csv")
    for val in value:
        df_intTT["cumulative{}NumberForDistrict".format(val)][idx] = df_state["cumulative{}NumberForState".format(val)][0]
        
for val in value:
    df_intTT["cumulative{}NumberForState".format(val)] = df_intTT["cumulative{}NumberForDistrict".format(val)].sum()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
def addVaccination(df,date):
    # df = df.drop(['cumulativeVaccinatedNumberForDistrict', 'cumulativeVaccinated1NumberForDistrict', 'cumulativeVaccinated2NumberForDistrict'], axis=1)
    with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(date)) as url:
        data = json.loads(url.read().decode())
    df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
    df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
    df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]
    states = []
    vaccinated = []
    vaccinated1 = []
    vaccinated2 = []
    for sta in data['getBeneficiariesGroupBy']:
        if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
            states.append('Dadra and Nagar Haveli and Daman and Diu')
        else:
            states.append(sta['title'])
        vaccinated.append(sta['total'])
        vaccinated1.append(sta['partial_vaccinated'])
        vaccinated2.append(sta['totally_vaccinated'])
    VAC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()
    VAC_df = VAC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})
    df = df.merge(VAC_df,how="left",on="District")
    return df

In [17]:
df_intTT = addVaccination(df_intTT,date)

In [18]:
def getDeltaValues(df,date):
    prevDate = date - timedelta.Timedelta(days=1)
    cs = pd.read_csv("../../RAWCSV/"+str(date)+"/TT_final.csv")
    df["deltaConfirmedForState"] = df["cumulativeConfirmedNumberForState"] - cs["cumulativeConfirmedNumberForState"][0]
    df["deltaDeceasedForState"] = df["cumulativeDeceasedNumberForState"] - cs["cumulativeDeceasedNumberForState"][0]
    df["deltaRecoveredForState"] = df["cumulativeRecoveredNumberForState"] - cs["cumulativeRecoveredNumberForState"][0]
    df["deltaTestedForState"] = df["cumulativeTestedNumberForState"] - cs["cumulativeTestedNumberForState"][0]
    return df

In [19]:
df_intTT = getDeltaValues(df_intTT,date)

In [20]:
#Delta Vacc

In [21]:
import urllib.request, json 
with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(date)) as url:
    data = json.loads(url.read().decode())

In [22]:
states = []
vaccinated = []
vaccinated1 = []
vaccinated2 = []
for sta in data['getBeneficiariesGroupBy']:
    if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
        states.append('Dadra and Nagar Haveli and Daman and Diu')
    else:
        states.append(sta['title'])
    vaccinated.append(sta['total'])
    vaccinated1.append(sta['partial_vaccinated'])
    vaccinated2.append(sta['totally_vaccinated'])

In [23]:
prevdayVACC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()

In [24]:
prevdayVACC_df = prevdayVACC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})

In [25]:
lst = ["Vaccinated1","Vaccinated2"]
for val in lst:
    df_intTT["delta{}ForDistrict".format(val)] = None
    for idx in df_intTT.index:
        df_intTT["delta{}ForDistrict".format(val)][idx] = df_intTT["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == df_intTT["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
cs = pd.read_csv("../../RAWCSV/"+str(date)+"/TT_final.csv")
states_df = cs[["Date","District","cumulativeConfirmedNumberForDistrict","cumulativeRecoveredNumberForDistrict","cumulativeDeceasedNumberForDistrict"]]

In [27]:
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    df_intTT["delta{}ForDistrict".format(val)] = None
    for idx in df_intTT.index:
        df_intTT["delta{}ForDistrict".format(val)][idx] = int(df_intTT["cumulative{}NumberForDistrict".format(val)][idx]) - int(states_df[(states_df["District"] == df_intTT["District"][idx]) & (states_df["Date"] == str(date))]["cumulative{}NumberForDistrict".format(val)].item())

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [28]:
lst = ["Vaccinated1","Vaccinated2"]
for val in lst:
    df_intTT["delta{}ForDistrict".format(val)] = None
    for idx in df_intTT.index:
        df_intTT["delta{}ForDistrict".format(val)][idx] = df_intTT["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == df_intTT["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [29]:
df_intTT["7DmaTestedForState"] = None
df_intTT["delta21_14confirmedForState"] = None

In [30]:
population = pd.read_csv("../../CSV/StatePopulation.csv")

In [31]:
population = population.merge(source,how="left",left_on = "State", right_on = "StateCode")

In [32]:
df_intTT["statePopulation"] = population[population["StateName"] == "India"]["StatePop"].item()

In [33]:
df_intTT = df_intTT.merge(population[["StateName","StatePop"]],how="left",left_on = "District", right_on = "StateName")

In [34]:
df_intTT = df_intTT.rename(columns={"StatePop":"districtPopulation"})

In [35]:
df_intTT.to_csv("../../RAWCSV/"+str(date)+"/TTint_final.csv",index=False)

In [36]:
from get7DMA import get_7dma_state
get_7dma_state("TT", str(date))

In [38]:
from BlanketUpdate import CreateJson,ts_json,ts_state_all

In [41]:
CreateJson(str(date))
ts_json("../../json_creation/Updated/data-"+str(date)+".min.json")
ts_state_all("../../json_creation/Updated/data-"+str(date)+".min.json")

In [ ]:
# AP
case1 : AP.raw is present >> run update derived value >> log.json

case2 : No AP.raw >> AP_final(Prev Day) and change date >> Do not run(Updatederived Value)

#TT_int
for all state files:
    for a state > Confirmed >> TT_raw_int(District Level)[Recoved,D,T,v1,v2] 
    
StateTotal == sum(DistrictCount)[C,R,D,T,V1,V2]

(TT_final_int)Delta = TT_raw_int - TT_final(day-1)

7dma(TT_final_int,interim = True)

JSON Creations